# Imports

In [59]:
import random
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

# Auxiliary methods

In [2]:
def Delta(I,J):
  if (I == J):
    return 1
  return 0
# Functions to randomly generate initial L-matrices
def RandMatrix(d):
  output = [[random.uniform(-10,10) for i in range(d)] for j in range(d)]
  return output
def RandMatrices(N, d):
  return [RandMatrix(d) for I in range(N)]

In [3]:
def ErrorMatrices(ells, arrs, N, d):
  output = [[
             np.matmul(ells[I],arrs[J]) + np.matmul(ells[J],arrs[I]) - 2*Delta(I,J)*np.identity(d)
             for J in range(N)] for I in range(N)]
  return output

In [4]:
# Functions for calculating total error (i.e. cost function)
def SquaredNorm(matrix):
  return np.trace(np.matmul(np.transpose(matrix), matrix))

def TotalError(ErrorMatrices, N):
  output = 0
  for I in range(N):
    for J in range(N):
      output += SquaredNorm(ErrorMatrices[I][J])
  return output

In [5]:
# Function for computing gradient w.r.t. components of L-matrices
def Gradient(ells, arrs, N, d):
  grad_L = [np.zeros((d,d)) for I in range(N)]
  grad_R = [np.zeros((d,d)) for I in range(N)]
  # Auxiliary quantity
  # We're repeatedly computing L_I*R_J here. Just compute it once!

  subaux = [[np.matmul(ells[I],arrs[J]) for J in range(N)] for I in range(N)]
  # Further optimization: compute the UR entries first, then copy to LL
  aux = [[subaux[I][J] + subaux[J][I] for J in range(N)] for I in range(N)]
  # Add 2nd-order terms
  for I in range(N):
    for i in range(d):
      for j in range(d):
        grad_L[I][i][j] -= 8*arrs[I][j][i]
        grad_R[I][i][j] -= 8*ells[I][j][i]
        # Add 4th-order terms
        for Jhat in range(N):
          for jhat in range(d):
            grad_L[I][i][j] += 4*aux[I][Jhat][i][jhat]*arrs[Jhat][j][jhat]
            grad_R[I][i][j] += 4*aux[I][Jhat][jhat][j]*ells[Jhat][jhat][i]


  return [grad_L, grad_R]

# Sanity check: Gradient should vanish at known solutions

In [ ]:
N = 4
d = 4

# Randomize L and R matrices
ells = RandMatrices(N,d)
arrs = RandMatrices(N,d)

# Initial L and R matrices at known solutions
ells[0] = [[1,0,0,0],[0,0,0,-1],[0,1,0,0],[0,0,-1,0]]
ells[1] = [[0,1,0,0],[0,0,1,0],[-1,0,0,0],[0,0,0,-1]]
ells[2] = [[0,0,1,0],[0,-1,0,0],[0,0,0,-1],[1,0,0,0]]
ells[3] = [[0,0,0,1],[1,0,0,0],[0,0,1,0],[0,1,0,0]]
ells = [ells[I] for I in range(N)]
for I in range(4):
  arrs[I] = np.transpose(ells[I])

# Compute cost function
error_matrices = ErrorMatrices(ells, arrs, N, d)
total_error = TotalError(error_matrices, N)
print("Total error = ", total_error)

# Compute gradient
[grad_L, grad_R] = Gradient(ells, arrs, N, d)
print("\nGrad_L[0] = \n", grad_L[0])
print("\nGrad_L[1] = \n", grad_L[1])
print("\nGrad_L[2] = \n", grad_L[2])
print("\nGrad_L[3] = \n", grad_L[3])
print("\nGrad_R[0] = \n", grad_R[0])
print("\nGrad_R[1] = \n", grad_R[1])
print("\nGrad_R[2] = \n", grad_R[2])
print("\nGrad_R[3] = \n", grad_R[3])

# Given 3 known 4x4 L and R matrix pairs, solve for the last pair.
## Here and below, picking a step size that gives reasonably quick convergence without overstepping early on is mostly done by manual experimentation.
## Unlike runs where all parameters are free to vary, this restricted gradient descent sometimes settles in non-global minima. If that happens, just re-run.

In [ ]:
N = 4
d = 4

# Randomize L and R matrices
ells = RandMatrices(N,d)
arrs = RandMatrices(N,d)

# Initial L and R matrices at known solutions (except the last one)
ells[0] = [[1.1,0.0,0.0,0.0],[0.0,0.0,0.0,-1.0],[0.0,1.0,0.0,0.0],[0.0,0.0,-1.0,0.0]]
ells[1] = [[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[-1.0,0.0,0.0,0.0],[0.0,0.0,0.0,-1.0]]
ells[2] = [[0.0,0.0,1.0,0.0],[0.0,-1.0,0.0,0.0],[0.0,0.0,0.0,-1.0],[1.0,0.0,0.0,0.0]]
for I in range(3):
  arrs[I] = np.transpose(ells[I])

# Set learning rate
epsilon = .0003

error_matrices = ErrorMatrices(ells, arrs, N, d)
total_error = TotalError(error_matrices, N)
print("Cost function: ", total_error)

# Gradient descent to find the last L and R matrices.
for count in range(15000):
  [grad_L, grad_R] = Gradient(ells, arrs, N, d)
  for i in range(d):
    for j in range(d):
      ells[3][i][j] -= epsilon*grad_L[3][i][j]
      arrs[3][i][j] -= epsilon*grad_R[3][i][j]

  error_matrices = ErrorMatrices(ells, arrs, N, d)
  total_error = TotalError(error_matrices, N)
  if (count % 250 == 0):
      print("Cost function: ", total_error)

print("\nL[3] = \n", ells[3])
print("\nR[3] = \n", arrs[3])


# Find 2 2x2 L and R matrices entirely from scratch.
## If you run this and find that it seems to get stuck, just add more steps and rerun (but of course comment out the RandMatrices() step first so the run doesn't restart). It sometimes hangs out for a while in regions of shallow gradient, but will keep descending if you wait.
## Here and elsewhere, the choice of step size is mostly from manual experimentation.

In [ ]:
N = 2
d = 2

# Randomize L and R matrices
ells = RandMatrices(N,d)
arrs = RandMatrices(N,d)

# Set learning rate
epsilon = .001

error_matrices = ErrorMatrices(ells, arrs, N, d)
total_error = TotalError(error_matrices, N)
print("Cost function: ", total_error)

# Gradient descent to find the last L and R matrices.
for count in range(10000):
  [grad_L, grad_R] = Gradient(ells, arrs, N, d)
  for I in range(N):
    for i in range(d):
      for j in range(d):
        ells[I][i][j] -= epsilon*grad_L[I][i][j]
        arrs[I][i][j] -= epsilon*grad_R[I][i][j]

  error_matrices = ErrorMatrices(ells, arrs, N, d)
  total_error = TotalError(error_matrices, N)
  if (count % 500 == 0):
      print("Cost function: ", total_error)


# Find 4 4x4 L and R matrices entirely from scratch.

In [ ]:
N = 4
d = 4

# Randomize L and R matrices
ells = RandMatrices(N,d)
arrs = RandMatrices(N,d)

# Set learning rate
epsilon = .000001

error_matrices = ErrorMatrices(ells, arrs, N, d)
total_error = TotalError(error_matrices, N)
print("Cost function: ", total_error)

# Gradient descent to find the last L and R matrices.
for count in range(100000):
  if (count == 10000):
    print("Increasing stepsize to 10e-4")
    epsilon = .0001
  if (count == 20000):
    print("Increasing stepsize to 3x10e-3")
    epsilon = .0003
  [grad_L, grad_R] = Gradient(ells, arrs, N, d)
  for I in range(N):
    for i in range(d):
      for j in range(d):
        ells[I][i][j] -= epsilon*grad_L[I][i][j]
        arrs[I][i][j] -= epsilon*grad_R[I][i][j]

  error_matrices = ErrorMatrices(ells, arrs, N, d)
  total_error = TotalError(error_matrices, N)
  if (count % 1000 == 0):
      print("Cost function: ", total_error)

# Finding 16 128x128 L and R matrices entirely from scratch
## The outputs here are novel, as previously the only 1D off-shell (N,d)=(16,128) representations in the literature were those generated by dimensional reduction of higher-D representations.
## Given the scale, we make a few optimizations here not present for smaller representations, namely:
*   Computing the gradient directly rather than by calling the auxiliary function above to avoid making excess copies.
*   Descending as far as possible before recomputing the gradient.
*   Gradual simulated annealing of the step size within each cycle so that we don't waste time making excessively small steps nor risk significant overstepping.

## In practice, we found that this took about 2500 steps and roughly 30 minutes to run, with a significant temporary slowdown around E = 7000.



In [ ]:
N = 16
d = 128

# Randomize L and R matrices
# ells = RandMatrices(N,d)
# arrs = RandMatrices(N,d)

# Set learning rate
epsilon = .000001
error_matrices = ErrorMatrices(ells, arrs, N, d)
total_error = TotalError(error_matrices, N)
print("Initial error: ", total_error)

for count in range(2500):

  # Compute gradient
  grad_L = [np.zeros((d,d)) for I in range(N)]
  grad_R = [np.zeros((d,d)) for I in range(N)]
  for I in range(N):
    for Iprime in range(N):
      grad_R[I] += 4*np.matmul(np.transpose(ells[Iprime]), error_matrices[Iprime][I])
      grad_L[I] += 4*np.matmul(error_matrices[Iprime][I], np.transpose(arrs[Iprime]))
  
  old_cost = total_error
  
  # To avoid excessively small steps, we let the step size gradually (but
  # exponentially) grow inside the loop.
  # Assuming optimal step size is similar from cycle to cycle, we initialize it
  # half of an order of magnitude below the final step size from the last cycle.
  epsilon = epsilon / 3.0

  print("Descending...")
  while True:
    # Inch along gradient until it stops decreasing.
    epsilon = epsilon*1.2;
    old_error = total_error
    for I in range(N):
      ells[I] -= epsilon*grad_L[I]
      arrs[I] -= epsilon*grad_R[I]
    error_matrices = ErrorMatrices(ells, arrs, N, d)
    total_error = TotalError(error_matrices, N)
    if (total_error >= old_error):
      print("Cost function: ", total_error)
      break